In [214]:
from Bio import Entrez
Entrez.email = 'A.N.Other@example.com'

import Bio
import mygene

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

import csv

In [215]:
# Initialising variables (will eventually be user inputted values)
PAM = 'TTTC'
species = 'human'
repeat = 'UAAUUUCUACUAAGUGUAGAU'
length_of_target_seq = 24
# ??? How to deal with different formatted CSVs? how should I expect the user to input gene ids?
file = open(r"C:\Users\ievut\OneDrive\Desktop\Oncogene Library Project\All 15K Oncogenes.csv", 'r')

In [216]:
# Querying exon ranges for each gene in CSV file
mg = mygene.MyGeneInfo()
exon_list = mg.querymany(file, scopes = 'symbol', fields = 'exons', species = species)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-15201...done.
Finished.
143 input query terms found dup hits:
	[('A2MP1', 2), ('ABCC13', 2), ('ABCC6P1', 2), ('ACTBP2', 2), ('ANXA2P2', 2), ('ATG12P1', 2), ('ATXN8
678 input query terms found no hit:
	['ï»¿A1BG', 'AARS', 'AASTH42', 'ACN9', 'ACPP', 'ACT', 'ADCK3', 'ADCK4', 'ADPRHL2', 'ADRBK1', 'ADRBK2
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [217]:
# Search for sequence of first variant in database
def fetch_details(accession_number_of_first_variant):
    handle = Entrez.efetch(db = "nucleotide", id = accession_number_of_first_variant, rettype = "gb", retmode = "text")
    details = SeqIO.read(handle, "genbank")
    handle.close()
    return details

In [218]:
valid_gene_dict = {}
found_target_seq_counter = 0

# Iterating through list of gene exon ranges acquired from previous query
for gene in exon_list:
    if 'exons' in gene:
        all_exon_ranges = gene['exons']
        accession_number_of_first_variant = (all_exon_ranges[0])['transcript']
    
    record = fetch_details(accession_number_of_first_variant)
   
    # Iterating through features of variant to find range of coding strand
    for feat in record.features:
        if feat.type == 'CDS':
            start_of_CDS_range_of_variant = feat.location.start
            break
            
    # Iterating through features of variant to find range of exon which contains start of coding strand
    for feat in record.features:
        if feat.type == 'exon' and feat.location.end > start_of_CDS_range_of_variant:
            exon_range_of_variant = feat.location
            break
   
    feature_location = FeatureLocation(start_of_CDS_range_of_variant, exon_range_of_variant.end)
    
    # Extracting the selected exon sequence
    exon_seq_to_extract = SeqFeature(feature_location)
    seq_to_search = exon_seq_to_extract.extract(record)

    # Searching from left of exon sequence
    start_pos = (seq_to_search.seq).find(PAM)

    # Converting target sequence with repeat sequence into gRNA
    # ??? How to deal with issue if there are less than 20 nucleotides left in sequence after the PAM ???
    target_seq_DNA = seq_to_search[start_pos:(start_pos + length_of_target_seq)] 
    target_seq_RNA = (target_seq_DNA.seq).transcribe()
    gRNA = repeat + target_seq_RNA
    
    # Add found genes which contain found target sequences into dictionary
    if start_pos != -1:
        valid_gene_dict.update({gene['query']:{'gRNA':gRNA}})
        found_target_seq_counter = found_target_seq_counter + 1
    
print (valid_gene_dict)
print ("Number of target sequences found " + str(found_target_seq_counter) + " out of 300")

{'ï»¿A1BG': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCCUGGUACUGUCACUGCCACU')}, 'A4GALT': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCGUCUCCAUCAUGAUCUACUG')}, 'A4GNT': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCAAGUCCCACCAGGGGCUGGA')}, 'AAA1': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCAAGUCCCACCAGGGGCUGGA')}, 'AAK1': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCGACUCCCGGCGAGAGCAGGG')}, 'AANAT': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCGCUGCCUCACCCCGGAGGAC')}, 'AARS': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCGCUGCCUCACCCCGGAGGAC')}, 'AARS2': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCUGAACUUCUUUCGGGACCGC')}, 'AASDHPPT': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCCCUGCCAAACGGUUCUGCUU')}, 'ABALON': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCCACGCACAGUGCCCCGCCGA')}, 'ABCA1': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCAGAAGAAGACAAACA')}, 'ABCA17P': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCGCAGCAGUGGAAGUCUUCAC')}, 'ABCA6': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCUUAAGAAAUGGAGGAUGAAA')}, 'ABCA7': {'gRNA': Seq('UAAUUUCUACUAAGUGUAGAUUUUCA